In [91]:
from datasets import load_dataset

ds = load_dataset("benjamin-paine/free-music-archive-small")

In [92]:
# ds = ds.with_format("polars")
# import polars
# import polars.datatypes

def has_all_genres(examples, genre_ids):
  return all(g in examples['genres'] for g in genre_ids)

def has_any_genres(examples, genre_ids):
  return any(g in examples['genres'] for g in genre_ids)


fds = ds['train'].filter(lambda x: has_any_genres(x, [12, 26, 58, 25]))

# ind = ds['train'][:].loc[:, (polars_column.map_elements(lambda x : all( g in [4, 26] for g in x) , return_dtype=polars.datatypes.Boolean) )]
# indices = [i for i, genres in enumerate(polars_column) if genres is not None and 70 in genres and 4 in genres]
print(f"Indices where genres contain both 70 and 4: {fds}")
# .filter(lambda x: print(x) == None).index

Indices where genres contain both 70 and 4: Dataset({
    features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
    num_rows: 929
})


In [93]:
# Get all unique genres from the dataset
all_genres = set()
for example in fds['genres']:
  if example:
    all_genres.update(example)

print(f"Number of unique genres: {len(all_genres)}")
print("All unique genres:", sorted(list(all_genres)))

Number of unique genres: 21
All unique genres: [6, 15, 23, 26, 27, 28, 39, 43, 45, 46, 51, 58, 60, 65, 73, 74, 96, 121, 136, 156, 158]


In [94]:
# Get all unique artists from the filtered dataset
artists = set()
for example in fds['artist']:
  artists.add(example)

print(f"Number of unique artists in filtered dataset: {len(artists)}")
print("Artists:", sorted(list(artists)))

Number of unique artists in filtered dataset: 212
Artists: ['20lb Sounds', 'Ai Aso', 'Albin Andersson', 'Alec K. Redfearn & the Eyesores', 'Alexander Turnquist', 'Allysen Callery', 'Alyiann', 'Amitron_7', 'Anamoli', 'Andrea Tomasi', 'Andreas Woiger', 'Anima', 'Beat Culture', 'Beat Doctor', 'Ben von Wildenhaus', 'Big Blood', 'Bill Borman', 'Bird of Youth', 'Boca Chica', "Bonnie 'Prince' Billy", 'Brett Saxon', 'Bridget St John', 'Brigid Mae Power', 'Brock Tyler', 'Broke For Free', 'Cal Folger Day', 'Carter Behnke', 'Cath and Phil Tyler', 'Charles Manson', 'Chuck Johnson', 'Cityzens Were Here', 'Colby Maddox', 'Connie Acher & Blind Drunk John', 'Connie Acher & Jelly', 'Curtis Eller', 'D SMILEZ', 'Dave Kent', 'David Rovics', 'David Szesztay', 'Debby Schwartz', 'Delgarma', 'Derek Clegg', 'Dick Whyte and his Golden Guitar', "Digi G'Alessio", 'Dora Bleu', 'Ed Askew', 'Ed Trickett', 'Entertainment for the Braindead', 'Eric Carbonara', 'Eric Lindley', 'Eszter Balint', 'Fast Freddie Fourier and 

In [68]:
# fds.save_to_disk("filtered_dataset")

In [95]:
import random

# Get list of all artists and set random seed for reproducibility
artist_list = list(artists)
random.seed(42)
random.shuffle(artist_list)

n_artist = len(artist_list)
train_size = int(0.7 * n_artist)
val_size = int(0.15 * n_artist)

train_artists = set(artist_list[:train_size])
val_artists = set(artist_list[train_size:train_size + val_size])
test_artists = set(artist_list[train_size + val_size:])

print(f"Train artists: {len(train_artists)}")
print(f"Val artists: {len(val_artists)}")
print(f"Test artists: {len(test_artists)}")

Train artists: 148
Val artists: 31
Test artists: 33


In [96]:
# Split the filtered dataset (fds) based on artist splits
train_split = fds.filter(lambda x: x['artist'] in train_artists)
val_split = fds.filter(lambda x: x['artist'] in val_artists)
test_split = fds.filter(lambda x: x['artist'] in test_artists)

print(f"Train split size: {len(train_split)}")
print(f"Val split size: {len(val_split)}")
print(f"Test split size: {len(test_split)}")
print(f"Total: {len(train_split) + len(val_split) + len(test_split)} (original fds: {len(fds)})")

Train split size: 686
Val split size: 119
Test split size: 124
Total: 929 (original fds: 929)


In [97]:
from datasets import Dataset, DatasetDict
filtered_ds = DatasetDict({"train": train_split, 
                                 "validation": val_split, 
                                 "test": test_split})
# filtered_ds['genres'] = Dataset.from_dict({"any": [12, 26, 58, 25]})
filtered_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [131]:

from diffusers import DiffusionPipeline
import torch
# from prefigure.prefigure import push_wandb_config
import torch
from torch import optim
from torch.nn import functional as F
import pytorch_lightning as pl
import torchaudio
import torchtune.training
import wandb
import librosa
import torchtune
from datasets import load_from_disk
from peft import LoraConfig, inject_adapter_in_model
from dataclasses import dataclass
# import torch_audiomentations as taug
import torchaudio as ta
import audiomentations as aug
import audiomentations as taug
import random
# import diffusers
import math
from torch.utils.data import Dataset
from pytorch_lightning.callbacks import RichProgressBar
import numpy as np

models_map = {
  "glitch-440k": {
    'sample_rate': 48000,
    'sample_size': 65536
    },
  "jmann-small-190k": {
    'sample_rate': 48000,
    'sample_size': 65536
    },
  "jmann-large-580k": {
    'sample_rate': 48000,
    'sample_size': 131072
    },
  "maestro-150k": {
    'sample_rate': 16000,
    'sample_size': 65536
    },
  "unlocked-250k": {
  'sample_rate': 16000,
  'sample_size': 65536
  },
  "honk-140k": {
    'sample_rate': 16000,
    'sample_size': 65536
    },
}

@dataclass
class TrainingConfig:
    model_name = "unlocked-250k"
    data_path = "./filter_fma_rock"
    sample_size = models_map[model_name]['sample_size']
    sample_rate = models_map[model_name]['sample_rate']
    data_loader_num_workers = 4
    batch_size = 32
    # eval_batch_size = 16  # how many images to sample during evaluation
    gradient_accumulation_steps = 1
    lora_rank = 16  # the rank of the LoRA layers1
    lora_alpha = 16 # scaling factor, which seems hardly necessary
    lr = 4e-5
    lr_warmup_steps = 5 # epochs
    num_cycles=0.5  # cosine annealing cycles
    num_epochs = 100
    # save_image_epochs = 10
    save_model_epochs = 30
    demo_every = 5
    n_samples = 10
    # save_demo = True
    demo_save_path = './demo_songs'
    ckpt_load_path = None # 'best', 'last', <path]>
    wandb_log_model = 'all'
    check_val_every_n_epoch = 5
    # mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = f"ddim-lora-{model_name}"  # the model name locally and on the HF Hub
    name = f"ddim-lora-{model_name}-{random.randint(0, 1000)}"  # the name of the wandb run
    project_name = "nn-audio-diffusion"
    save_path = f'{name}-ckpt'
    # overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    # seed = 42

config = TrainingConfig()

def optional(x, bool):
  if bool:
    return [x]
  else:
    return []

# samples_per_s = 44100
def audio_augmentations(sample_size, use_train_augs = False):
  output_type = 'tensor'
  augmentation = aug.Compose(
    transforms=
      optional( # can change length, therefore first
        aug.RepeatPart(mode="insert", p=0.8)
        , use_train_augs
      )
      +
      [aug.AdjustDuration(duration_samples=sample_size, p=1.0)]
      +
      optional(
        aug.TimeStretch(
            min_rate=0.8,
            max_rate=1.25,
            leave_length_unchanged=True,
            p=0.8
        ),
        use_train_augs)
      + optional(
        aug.OneOf(
          transforms=[
              taug.LowPassFilter(
                  min_cutoff_freq=500.0,
                  max_cutoff_freq=2000.0,
                  p=1.0,
                  # output_type=output_type
              ),
              taug.HighPassFilter(
                  min_cutoff_freq=100.0,
                  max_cutoff_freq=2400.0,
                  p=1.0,
                  # output_type=output_type
              ),
              taug.BandPassFilter(
                  # min_center_frequency=200.0,
                  # max_center_frequency=4000.0,
                  min_center_freq=200.0,
                  max_center_freq=4000.0,
                  p=1.0,
                  # output_type=output_type
              ),
              aug.ClippingDistortion(min_percentile_threshold=0,
                  max_percentile_threshold=30,
                  p=0.5),
              # aug.Limiter(
              #   min_threshold_db=-16.0,
              #   max_threshold_db=0.0,
              #   threshold_mode="relative_to_signal_peak",
              #   p=1.0,
              # ), # todo: cylimiter dependency problem
              taug.PitchShift(
                # min_transpose_semitones=-5.0,
                # max_transpose_semitones=5.0,
                min_semitones=-5.0,
                max_semitones=5.0,
                # sample_rate=sample_rate,
                p=1.0,
                # output_type=output_type
              ),
          ],
          p=1.0
        ),
        use_train_augs)
      +
      optional(
        aug.SomeOf( transforms=[
          aug.TanhDistortion(
            min_distortion=0.01,
            max_distortion=0.15,
            p=0.8
          )
          # ,
          # aug.RepeatPart(mode="insert", p=0.8),
        ],
        num_transforms=(0,None),
        ),
        use_train_augs)
      +
      optional(
        aug.Gain(
          min_gain_db=-15.0,
          max_gain_db=5.0,
          # p=0.5,
          p=0.5,
        )
        , use_train_augs)
      +
      # optional(
      #   taug.ShuffleChannels(p=0.5, 
      #     # output_type=output_type
      #     ),
      #   use_train_augs)
      # +
      
      # taug.OneOf(transforms=[
      #   taug.PeakNormalization(apply_to="only_too_loud_sounds", p=1.0, output_type=output_type),
        # [taug.PeakNormalization(apply_to="all", p=1.0
        #                         # , output_type=output_type
        #                         )]
        [aug.Normalize(apply_to="only_too_loud_sounds", p=1.0)]
      # ], output_type=output_type),
      +
      optional(
        aug.PolarityInversion(p=0.5),
      use_train_augs)
  )

  # def transforms(examples):
  #   # print(len(examples['audio']), "audio samples")
  #   x = [augmentation(aud['array'], sample_rate=aud['sampling_rate']) for aud in examples['audio']]
  #   examples["audio"] = x
  #   print("Transformed audio samples:", [aud.shape for aud in examples['audio']])
  #   return examples
  
  # return transforms, augmentation
  return augmentation


In [28]:
# aug = audio_augmentations(config.sample_size, config.sample_rate, use_train_augs=True)
# aug(np.zeros((config.sample_size,)), sample_rate=config.sample_rate)

In [127]:
import torch
config.data_loader_num_workers = 1

class MusicDataset(Dataset):
  def __init__(self, hf_dataset, transform=None):
    self.dataset = hf_dataset
    self.transform = transform
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    item = self.dataset[idx]
    audio = item['audio']['array']
    # if self.transform:
    print("sampling_rate: ", item['audio']['sampling_rate'])
    audio_tensor = self.transform(audio, sample_rate=item['audio']['sampling_rate'])
    audio_tensorr = torch.tensor(audio_tensor, dtype=torch.float32)
    print("transform:", audio_tensor.shape)
    print("tensor", audio_tensorr.shape)
    return audio_tensor

def make_dataloaders(data, config=config):
  sample_size=config.sample_size
  # sample_rate=config.sample_rate
  num_workers=config.data_loader_num_workers
  batch_size=config.batch_size
  test_transformation = audio_augmentations(sample_size, use_train_augs = False)
  train_dataset = MusicDataset(data['train'], transform=audio_augmentations(sample_size, use_train_augs = True))
  val_dataset = MusicDataset(data['validation'], transform=test_transformation)
  test_dataset = MusicDataset(data['test'], transform=test_transformation)

  dl = lambda ds: torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)

  return {
    'train': dl(train_dataset),
    'validation': dl(val_dataset),
    'test': dl(test_dataset),
    'genres': data['genres'],
  }


In [128]:
this_data = load_from_disk("./rock_dataset_resampled")
this_data

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [129]:
daloaders = make_dataloaders(this_data, config=config)

In [130]:
for x in daloaders['train']:
  print(x.shape)

sampling_rate:  16000


/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  16000
transform: (65536,)
tensor torch.S

KeyboardInterrupt: 

In [100]:
for x in filtered_ds['train']:
  if x["audio"]["sampling_rate"] != 44100:
    print(x["audio"]["sampling_rate"])
    break

48000


In [ ]:
def make_datasets(data, config=config):
  sample_size=config.sample_size
  # config.sample_rate
  # num_workers=config.data_loader_num_workers
  # batch_size=config.batch_size
  trs = lambda audio, sample_rate: torchaudio.functional.resample(
    waveform = torch.tensor(audio, dtype=torch.float32), 
    orig_freq = sample_rate, 
    new_freq=config.sample_rate
  ).numpy()
  train_dataset = data['train'].map(transform=trs)
  val_dataset = MusicDataset(data['validation'], transform=trs)
  test_dataset = MusicDataset(data['test'], transform=trs)
  return {
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset,
    'genres': data['genres'],
  }



In [101]:
def map_audio(dataset, map_fn):
  """
  Applies a mapping function to the 'audio' column of a Hugging Face dataset.

  Args:
    dataset (datasets.Dataset): The input dataset.
    map_fn (callable): The function to apply to each audio object.
               It takes one argument (the audio object dict which contains 'array', 'sampling_rate', etc.)
               and returns the transformed audio data.

  Returns:
    datasets.Dataset: The dataset with the transformed 'audio' column.
  """
  return dataset.map(lambda example: {"audio": map_fn(example["audio"])})

In [102]:
# filtered_ds.pop('genres')
filtered_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [103]:
def trs(audio):
  waveform = torchaudio.functional.resample(
    waveform = torch.tensor(audio['array'], dtype=torch.float32), 
    orig_freq = audio['sampling_rate'], 
    new_freq=config.sample_rate
  ).numpy()
  audio["array"] = waveform
  audio["sampling_rate"] = config.sample_rate
  return audio
  
mapped_ds = map_audio(filtered_ds, trs)
mapped_ds

Map:   0%|          | 0/686 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [107]:
for x in mapped_ds['train']:
  print(x)
  break

{'audio': {'path': None, 'array': array([0.        , 0.        , 0.        , ..., 0.12091064, 0.16314697,
       0.08712769], shape=(479626,)), 'sampling_rate': 16000}, 'title': 'Queen Of The Wires', 'url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/The_Blind_Spot/Queen_Of_The_Wires', 'artist': 'Alec K. Redfearn & the Eyesores', 'composer': '', 'lyricist': '', 'publisher': '', 'genres': [58], 'tags': [], 'released': datetime.datetime(2008, 11, 26, 1, 44, 7), 'language': 'en', 'listens': 1299, 'artist_url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/', 'artist_website': 'http://www.aleckredfearn.com', 'album_title': 'The Blind Spot', 'album_url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/The_Blind_Spot/', 'license': 13, 'copyright': '', 'explicit': None, 'instrumental': 0, 'allow_commercial_use': 0, 'allow_derivatives': 0, 'require_attribution': 1, 'require_share_alike': 0}


In [104]:
# def aahhh(audio):
#   audio["sampling_rate"] = config.sample_rate
#   return audio
  
# mapped_ds = map_audio(filtered_ds, aahhh)
# mapped_ds

In [108]:
print(len(mapped_ds['train']), len(mapped_ds['validation']), len(mapped_ds['test']))
for x in mapped_ds['train']:
  print(x)
  break
for x in mapped_ds['test']:
  assert x['audio']['sampling_rate'] == 16000
for x in mapped_ds['validation']:
  assert x['audio']['sampling_rate'] == 16000
for x in mapped_ds['train']:
  assert x['audio']['sampling_rate'] == 16000

686 119 124
{'audio': {'path': None, 'array': array([0.        , 0.        , 0.        , ..., 0.12091064, 0.16314697,
       0.08712769], shape=(479626,)), 'sampling_rate': 16000}, 'title': 'Queen Of The Wires', 'url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/The_Blind_Spot/Queen_Of_The_Wires', 'artist': 'Alec K. Redfearn & the Eyesores', 'composer': '', 'lyricist': '', 'publisher': '', 'genres': [58], 'tags': [], 'released': datetime.datetime(2008, 11, 26, 1, 44, 7), 'language': 'en', 'listens': 1299, 'artist_url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/', 'artist_website': 'http://www.aleckredfearn.com', 'album_title': 'The Blind Spot', 'album_url': 'http://freemusicarchive.org/music/Alec_K_Redfearn_and_the_Eyesores/The_Blind_Spot/', 'license': 13, 'copyright': '', 'explicit': None, 'instrumental': 0, 'allow_commercial_use': 0, 'allow_derivatives': 0, 'require_attribution': 1, 'require_share_alike': 0}


In [109]:
mapped_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [111]:
from datasets import Dataset
mapped_ds['genres'] = Dataset.from_dict({"any": [12, 26, 58, 25]})

In [112]:
mapped_ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 686
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 119
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [ ]:
mapped_ds

In [113]:
mapped_ds.save_to_disk("rock_dataset_resampled")

Saving the dataset (0/2 shards):   0%|          | 0/686 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/119 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/124 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

In [52]:
len(filtered_ds['train']), len(filtered_ds['validation']), len(filtered_ds['test'])

(686, 119, 124)

In [53]:
from datasets import Dataset
filtered_sampled_ds = Dataset.from_dict(make_datasets(filtered_ds, config=config))
filtered_sampled_ds

AttributeError: 'MusicDataset' object has no attribute '_indices'

In [ ]:
dls = make_dataloaders(filtered_ds, config)
for x in dls['train']:
  print(x)

sampling_rate:  44100


/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (120904,)
tensor torch.Size([120904])
sampling_rate:  44100
transform: (114143,)
tensor torch.Size([114143])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (76901,)
tensor torch.Size([76901])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (171601,)
tensor torch.Size([171601])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor t

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/uti

 (82801,)
tensor torch.Size([82801])
sampling_rate:  44100


transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (157999,)
tensor torch.Size([157999])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (96663,)
tensor torch.Size([96663])
sampling_rate:  44100
transform: (93734,)
tensor torch.Size([93734])
sampling_rate:  44100
transform: (188149,)
tensor torch.Size([188149])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (65536,)
tensor torch.Size([65536])
sampling_rate:  44100
transform: (210316,)
tensor torch.Size([210316])
sampling_rate:  44100
transform: (65536,)
tensor t

In [77]:
filtered_ds.save_to_disk("filtered_dataset_by_artist_split")

Saving the dataset (0/2 shards):   0%|          | 0/653 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/152 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/124 [00:00<?, ? examples/s]

In [2]:
from datasets import load_from_disk

def load_dataloader(path):
  ds = load_from_disk(path)
  ds['train'].set_transform()


pds = load_from_disk("filtered_dataset_by_artist_split")
print(f"Loaded dataset: {pds}")

Loaded dataset: DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 653
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 152
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'arti

In [ ]:
# load_from_disk("filtered_fma_rock")
from datasets import Dataset
pds['genres'] = Dataset.from_dict({"any": [26, 58, 25]})

In [9]:
pds

DatasetDict({
    train: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 653
    })
    validation: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
        num_rows: 152
    })
    test: Dataset({
        features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist

In [10]:
pds.save_to_disk("filter_fma_rock")

Saving the dataset (0/2 shards):   0%|          | 0/653 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/152 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/124 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

In [3]:
pds = load_from_disk("filtered_dataset_by_artist_split")
pds['train']

Dataset({
    features: ['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'],
    num_rows: 653
})

In [12]:
xs = [1,2,3]
xs + [4,5,6]

[1, 2, 3, 4, 5, 6]

In [ ]:
# import torch_audiomentations as taug
import torchaudio as ta
import audiomentations as aug
import audiomentations as taug
from datasets import Features, Audio

def optional(x, bool):
  if bool:
    return [x]
  else:
    return []

# samples_per_s = 44100
def audio_augmentations(sample_size, sample_rate, use_train_augs = False):
  output_type = 'tensor'
  augmentation = aug.Compose(
    transforms=
      [aug.AdjustDuration(duration_samples=sample_size, p=1.0)]
      +
      optional(
        aug.TimeStretch(
            min_rate=0.8,
            max_rate=1.25,
            leave_length_unchanged=True,
            p=0.8
        ),
        use_train_augs)
      + optional(
        aug.OneOf(
          transforms=[
              taug.LowPassFilter(
                  min_cutoff_freq=500.0,
                  max_cutoff_freq=2000.0,
                  p=1.0,
                  # output_type=output_type
              ),
              taug.HighPassFilter(
                  min_cutoff_freq=100.0,
                  max_cutoff_freq=2400.0,
                  p=1.0,
                  # output_type=output_type
              ),
              taug.BandPassFilter(
                  # min_center_frequency=200.0,
                  # max_center_frequency=4000.0,
                  min_center_freq=200.0,
                  max_center_freq=4000.0,
                  p=1.0,
                  # output_type=output_type
              ),
              aug.ClippingDistortion(min_percentile_threshold=0.01,
                  max_percentile_threshold=0.99,
                  p=0.5),
              aug.Limiter(
                min_threshold_db=-16.0,
                max_threshold_db=-6.0,
                threshold_mode="relative_to_signal_peak",
                p=1.0,
              ),
              taug.PitchShift(
                # min_transpose_semitones=-5.0,
                # max_transpose_semitones=5.0,
                min_semitones=-5.0,
                max_semitones=5.0,
                # sample_rate=sample_rate,
                p=1.0,
                # output_type=output_type
              ),
          ],
          p=1.0
        ),
        use_train_augs)
      +
      optional(
        aug.SomeOf( transforms=[
          aug.TanhDistortion(
            min_distortion=0.01,
            max_distortion=0.15,
            p=0.8
          ),
          aug.RepeatPart(mode="insert", p=0.8),
        ],
        num_transforms=(0,None),
        ),
        use_train_augs)
      +
      optional(
        aug.Gain(
          min_gain_db=-15.0,
          max_gain_db=5.0,
          # p=0.5,
          p=0.5,
        )
        , use_train_augs)
      +
      # optional(
      #   taug.ShuffleChannels(p=0.5, 
      #     # output_type=output_type
      #     ),
      #   use_train_augs)
      # +
      
      # taug.OneOf(transforms=[
      #   taug.PeakNormalization(apply_to="only_too_loud_sounds", p=1.0, output_type=output_type),
        # [taug.PeakNormalization(apply_to="all", p=1.0
        #                         # , output_type=output_type
        #                         )]
        [aug.Normalize(apply_to="only_too_loud_sounds", p=1.0)]
      # ], output_type=output_type),
      +
      optional(
        aug.PolarityInversion(p=0.5),
      use_train_augs)
  )

  # def transforms(examples):
  #   # print(len(examples['audio']), "audio samples")
  #   x = [augmentation(aud['array'], sample_rate=aud['sampling_rate']) for aud in examples['audio']]
  #   examples["audio"] = x
  #   print("Transformed audio samples:", [aud.shape for aud in examples['audio']])
  #   return examples
  
  # return transforms, augmentation
  return augmentation


In [168]:
sample_rate = 16000
sample_size = 65536
# taug, aug = audio_augmentations(sample_size, sample_rate, use_train_augs = False)
taug, aug = audio_augmentations(sample_size, sample_rate, use_train_augs = False)

TypeError: cannot unpack non-iterable Compose object

In [170]:
pds = load_from_disk("./filter_fma_rock")

In [171]:
import torch
from torch.utils.data import Dataset

class MusicDataset(Dataset):
  def __init__(self, hf_dataset, transform=None):
    self.dataset = hf_dataset
    self.transform = transform
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    item = self.dataset[idx]
    audio = item['audio']['array']
    if self.transform:
      audio = self.transform(audio, sample_rate=item['audio']['sampling_rate'])
    audio_tensor = torch.tensor(audio, dtype=torch.float32)
    return audio_tensor

test_transformation = audio_augmentations(sample_size, sample_rate, use_train_augs = False)
train_dataset = MusicDataset(pds['train'], transform=audio_augmentations(sample_size, sample_rate, use_train_augs = True))
val_dataset = MusicDataset(pds['validation'], transform=test_transformation)
test_dataset = MusicDataset(pds['test'], transform=test_transformation)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 653
Val dataset size: 152
Test dataset size: 124


In [ ]:
def make_dataloaders(data, num_workers=4, batch_size=32):
  test_transformation = audio_augmentations(sample_size, sample_rate, use_train_augs = False)
  train_dataset = MusicDataset(data['train'], transform=audio_augmentations(sample_size, sample_rate, use_train_augs = True))
  val_dataset = MusicDataset(data['validation'], transform=test_transformation)
  test_dataset = MusicDataset(data['test'], transform=test_transformation)

  dl = lambda ds: data.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)

  return {
    'train': dl(train_dataset),
    'validation': dl(val_dataset),
    'test': dl(test_dataset)
  }

In [182]:
from torch.utils import data
# pds['train'].set_transform(aug)
# pds['train'].set_format("torch", columns=["audio"])

train_dl = data.DataLoader(train_dataset, 16, shuffle=False, num_workers=1) # num_workers=2 is enough for 2 shards

In [ ]:
for i in train_dl:
  print(i.shape)
  break

/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
Failed to import cylimiter. Maybe it is not installed? To install the optional cylimiter dependency of audiomentations, run `pip install cylimiter` or `pip install audiomentations[extras]`


ModuleNotFoundError: Caught ModuleNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/nix-shell-6340-0/ipykernel_8777/3023671263.py", line 16, in __getitem__
    audio = self.transform(audio, sample_rate=item['audio']['sampling_rate'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/audiomentations/core/c

Failed to import cylimiter. Maybe it is not installed? To install the optional cylimiter dependency of audiomentations, run `pip install cylimiter` or `pip install audiomentations[extras]`


Failed to import cylimiter. Maybe it is not installed? To install the optional cylimiter dependency of audiomentations, run `pip install cylimiter` or `pip install audiomentations[extras]`


In [174]:
for i in val_dataset:
  print(i)
  break

tensor([-0.0445, -0.0464, -0.0459,  ..., -0.0213, -0.0375, -0.0555])


In [162]:
def auggg(aud):
  return aug(aud['array'])
pds['train']['audio'].map(auggg)

AttributeError: 'list' object has no attribute 'map'

In [146]:
pds['train'] = pds['train'].with_format("torch", columns=["audio"])
pds['train'].set_transform(aug)
for batch in pds['train']:
  # print(batch)
  # print(batch['audio'])
  print(type(batch['audio_sample'][0]))
  break

Transformed audio samples: [(65536,)]


/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:107: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


KeyError: 'audio_sample'

In [136]:
pds['train'].

SyntaxError: invalid syntax (3068763972.py, line 1)

In [ ]:
from torch.utils import data
pds['train'].set_transform(aug)
pds['train'].set_format("torch", columns=["audio"])

train_dl = data.DataLoader(pds['train'], 10, shuffle=True, num_workers=1, drop_last=True, col) # num_workers=2 is enough for 2 shards
len(train_dl)

65

In [157]:
for batch in train_dl:
  print(batch)
  break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/zmrocze/studia/uwr/sem2/nn/nn_projekt/.mamba/envs/my_env1/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
TypeError: 'bool' object is not callable


In [1]:
pds = load_from_disk("filtered_dataset_by_artist_split")
# pds = pds.with_format("torch")
# pds['train'].set_transform(
#   audio_augmentations(sample_size=65536, sample_rate=44100)
# )

NameError: name 'load_from_disk' is not defined

In [68]:
from torch.utils import data
train_dl = data.DataLoader(pds['train'], 10, shuffle=True)


In [69]:
first_batch = next(iter(train_dl))
print(first_batch)

RuntimeError: stack expects each tensor to be equal size, but got [1321967] at entry 0 and [1323119] at entry 4

In [58]:
import torch

# Get first batch from train_dl and play the first audio sample
import IPython.display as ipd

# Get first batch
first_batch = next(iter(train_dl))

print(first_batch)

# Extract the first audio sample from the batch
first_audio = first_batch['audio_sample'][0]  # Get first sample from batch
sample_rate = 44100  # Using the sample rate from your augmentation function

print(first_audio)

# Play the audio
ipd.Audio(first_audio, rate=sample_rate)

dict_keys(['audio', 'title', 'url', 'artist', 'composer', 'lyricist', 'publisher', 'genres', 'tags', 'released', 'language', 'listens', 'artist_url', 'artist_website', 'album_title', 'album_url', 'license', 'copyright', 'explicit', 'instrumental', 'allow_commercial_use', 'allow_derivatives', 'require_attribution', 'require_share_alike'])


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# pds = pds.set_transform(transforms)
pds['train'].set_transform(transforms)
pds
# fds = fds.with_format("torch")

In [2]:
ds['train'][0]

{'audio': {'path': '000002.mp3',
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.24734139,
         -0.19890546, -0.10515908], shape=(1321967,)),
  'sampling_rate': 44100},
 'title': 'Food',
 'url': 'http://freemusicarchive.org/music/AWOL/AWOL_-_A_Way_Of_Life/Food',
 'artist': 'AWOL',
 'composer': '',
 'lyricist': '',
 'publisher': '',
 'genres': [70],
 'tags': [],
 'released': datetime.datetime(2008, 11, 26, 1, 48, 12),
 'language': 'en',
 'listens': 1293,
 'artist_url': 'http://freemusicarchive.org/music/AWOL/',
 'artist_website': 'http://www.AzillionRecords.blogspot.com',
 'album_title': 'AWOL - A Way Of Life',
 'album_url': 'http://freemusicarchive.org/music/AWOL/AWOL_-_A_Way_Of_Life/',
 'license': 18,
 'copyright': '',
 'explicit': 1,
 'instrumental': 0,
 'allow_commercial_use': 0,
 'allow_derivatives': 1,
 'require_attribution': 1,
 'require_share_alike': 1}

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("audio-classification", model="dima806/music_genres_classification")

config.json:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import torch
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

In [ ]:
# def filter_by_genres(dataset, genre_ids):
#   def contains_genres(example):
#     print(example)
#     if example is None:
#       return False
#     return False
#     return all(genre_id in example['genres'] for genre_id in genre_ids)
  
#   return .filter(contains_genres)

ds


filter_by_genres(ds, [26, 4])

SyntaxError: invalid syntax (941091042.py, line 11)

In [ ]:
using polars (dataframe-like) interface for ds hugging face dataset perform given tasks: split data into train and test, putting 